In [ ]:
from google.colab import drive
from tqdm import tqdm_notebook as tqdm

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import matplotlib.pyplot as plt
from collections import Counter as C

In [ ]:
%cd /content/gdrive/MyDrive/'10701-project'

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/10701'
/content


In [ ]:
def text_process(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

## Classifiers

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
def rf_clf(x, y):
    rmfr = RandomForestClassifier(random_state=101)
    rmfr.fit(x,y)
    return rmfr

def dt_clf(x,y):
    dt = DecisionTreeClassifier(random_state=101)
    dt.fit(x,y)
    return dt

def svm_clf(x,y):
    svm = SVC(random_state=101)
    svm.fit(x,y)
    return svm
    
def gb_clf(x,y):
    gbi = GradientBoostingClassifier(learning_rate=0.1,max_depth=5,max_features=0.5,random_state=999999)
    gbi.fit(x,y)
    return gbi

def knn_clf(x,y):
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(x,y)
    return knn

def mlp_clf(x,y):
    mlp = MLPClassifier()
    mlp.fit(x,y)
    return mlp

## Text only Models

In [ ]:
import pandas as pd
### For 5-star just change the data 
train_data = pd.read_csv("./binary_data/train.csv")
test_data = pd.read_csv("./binary_data/test.csv")

In [ ]:
x = train_data.values[:, 2]
y = train_data.values[:, 0]

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk.corpus import stopwords
vocab = TfidfVectorizer(analyzer=text_process).fit(x)

In [ ]:
x = vocab.transform(x)
print("Shape of the sparse matrix: ", x.shape)
print("Non-Zero occurences: ",x.nnz)
density = (x.nnz/(x.shape[0]*x.shape[1]))*100
print("Density of the matrix = ",density)

Shape of the sparse matrix:  (31188, 62243)
Non-Zero occurences:  1622860
Density of the matrix =  0.08359937174911923


In [ ]:
x_test = test_data.values[:, 2]
y_test = test_data.values[:, 0]
x_test = vocab.transform(x_test)
#Shape of the matrix:
print("Shape of the sparse matrix: ", x_test.shape)
#Non-zero occurences:
print("Non-Zero occurences: ",x_test.nnz)

# DENSITY OF THE MATRIX
density = (x_test.nnz/(x_test.shape[0]*x_test.shape[1]))*100
print("Density of the matrix = ",density)

Shape of the sparse matrix:  (5504, 62243)
Non-Zero occurences:  281132
Density of the matrix =  0.08206185696047263


In [ ]:
rmfr = rf_clf(x,y)
predrmfr = rmfr.predict(x_test)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:",classification_report(y_test,predrmfr))

In [ ]:
dt = dt_clf(x,y)
preddt = dt.predict(x_test)
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(y_test,preddt))
print("Score:",round(accuracy_score(y_test,preddt)*100,2))
print("Classification Report:",classification_report(y_test,preddt))

In [ ]:
svm = svm_clf(x,y)
predsvm = svm.predict(x_test)
print("Confusion Matrix for Support Vector Machines:")
print(confusion_matrix(y_test,predsvm))
print("Score:",round(accuracy_score(y_test,predsvm)*100,2))
print("Classification Report:",classification_report(y_test,predsvm))

In [ ]:
gbi = gb_clf(x,y)
predgbi = gbi.predict(x_test)
print("Confusion Matrix for Gradient Boosting Classifier:")
print(confusion_matrix(y_test,predgbi))
print("Score:",round(accuracy_score(y_test,predgbi)*100,2))
print("Classification Report:",classification_report(y_test,predgbi))

In [ ]:
knn = knn_clf(x,y)
predknn = knn.predict(x_test)
print("Confusion Matrix for K Neighbors Classifier:")
print(confusion_matrix(y_test,predknn))
print("Score: ",round(accuracy_score(y_test,predknn)*100,2))
print("Classification Report:")
print(classification_report(y_test,predknn))

In [ ]:
mlp = mlp_clf(x,y)
predmlp = mlp.predict(x_test)
print("Confusion Matrix for Multilayer Perceptron Classifier:")
print(confusion_matrix(y_test,predmlp))
print("Score:",round(accuracy_score(y_test,predmlp)*100,2))
print("Classification Report:")
print(classification_report(y_test,predmlp))

## Meta and All features classification

In [ ]:
import pickle 
import string
from nltk.corpus import stopwords
import nltk 
nltk.download('stopwords')
## Preloaded tfidf vectorizer
with open("vectorizer.pkl", "rb") as fin:
  vocab = pickle.load(fin)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
## All features
x_train_feat = train_data[["useful", "funny", "cool", "date"]].values
x_train_cats = np.array([[int(x) for x in y.replace("[", "").replace("]", "").replace("\n", "").split()] for y in train_data["cat_vec"].values])
x_train = np.concatenate((x_train_feat, x_train_cats), axis = 1)

x_test_feat = test_data[["useful", "funny", "cool", "date"]].values
x_test_cats = np.array([[int(x) for x in y.replace("[", "").replace("]", "").replace("\n", "").split()] for y in test_data["cat_vec"].values])
x_test = np.concatenate((x_test_feat, x_test_cats), axis = 1)

In [ ]:
from scipy import sparse
x_train_complete = sparse.hstack([x,x_train])  
x_test_complete = sparse.hstack([x,x_test])  

In [ ]:
# Use _complete for complete data
rmfr = rf_clf(x_train, y)
predrmfr = rmfr.predict(x_test)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:",classification_report(y_test,predrmfr))

In [ ]:
# Use _complete for complete data
dt = dt_clf(x_train, y)
preddt = dt.predict(x_test)
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(y_test,preddt))
print("Score:",round(accuracy_score(y_test,preddt)*100,2))
print("Classification Report:",classification_report(y_test,preddt))

In [ ]:
# Use _complete for complete data
svm = svm_clf(x_train, y)
predsvm = svm.predict(x_test)
print("Confusion Matrix for Support Vector Machines:")
print(confusion_matrix(y_test,predsvm))
print("Score:",round(accuracy_score(y_test,predsvm)*100,2))
print("Classification Report:",classification_report(y_test,predsvm))

In [ ]:
# Use _complete for complete data
gbi = gb_clf(x_train, y)
predgbi = gbi.predict(x_test)
print("Confusion Matrix for Gradient Boosting Classifier:")
print(confusion_matrix(y_test,predgbi))
print("Score:",round(accuracy_score(y_test,predgbi)*100,2))
print("Classification Report:",classification_report(y_test,predgbi))

In [ ]:
# Use _complete for complete data
knn = knn_clf(x_train, y)
predknn = knn.predict(x_test)
print("Confusion Matrix for K Neighbors Classifier:")
print(confusion_matrix(y_test,predknn))
print("Score: ",round(accuracy_score(y_test,predknn)*100,2))
print("Classification Report:")
print(classification_report(y_test,predknn))

In [ ]:
# Use _complete for complete data
mlp = mlp_clf(x_train, y)
predmlp = mlp.predict(x_test)
print("Confusion Matrix for Multilayer Perceptron Classifier:")
print(confusion_matrix(y_test,predmlp))
print("Score:",round(accuracy_score(y_test,predmlp)*100,2))
print("Classification Report:")
print(classification_report(y_test,predmlp))